In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt

from tcnAutoencoder import TCNAutoencoder

In [2]:
# %load load_data.py
import pandas as pd
import glob, os, re


# Read the data only once.  It's big!
csv_files = glob.glob(os.path.join(".", "data", "hft_data", "*", "*_message_*.csv"))
date_str = re.compile(r'_(\d{4}-\d{2}-\d{2})_')
stock_str = re.compile(r'([A-Z]+)_\d{4}-\d{2}-\d{2}_')

df_list = []
day_list = []
sym_list = []

for csv_file in sorted(csv_files):
    date = date_str.search(csv_file)
    date = date.group(1)
    day_list.append(date)

    symbol = stock_str.search(csv_file)
    symbol = symbol.group(1)
    sym_list.append(symbol)

    # Find the order book file that matches this message file.
    book_file = csv_file.replace("message", "orderbook")

    # Read the message file and index by timestamp.
    df = pd.read_csv(csv_file, names=['Time','EventType','OrderID','Size','Price','Direction'])
    df['Time'] = pd.to_datetime(date) + pd.to_timedelta(df['Time'], unit='s')

    # Read the order book file and merge it with the messages.
    names = [f"{x}{i}" for i in range(1,11) for x in ["AP","AS","BP","BS"]]
    df = df.join(pd.read_csv(book_file, names=names), how='inner')
    df = df.set_index(['Time'])

    BBID_COL = df.columns.get_loc("BP1")
    BASK_COL = df.columns.get_loc("AP1")

    print (f"Read {df.shape[0]} unique order book shapshots from {csv_file}")

    df_list.append(df)

days = len(day_list)

Read 738034 unique order book shapshots from ./data/hft_data/AAPL/AAPL_2024-03-01_34200000_57600000_message_10.csv
Read 1923409 unique order book shapshots from ./data/hft_data/AAPL/AAPL_2024-03-04_34200000_57600000_message_10.csv
Read 2108353 unique order book shapshots from ./data/hft_data/AAPL/AAPL_2024-03-05_34200000_57600000_message_10.csv
Read 2364167 unique order book shapshots from ./data/hft_data/AAPL/AAPL_2024-03-06_34200000_57600000_message_10.csv
Read 1732063 unique order book shapshots from ./data/hft_data/AAPL/AAPL_2024-03-07_34200000_57600000_message_10.csv
Read 3123866 unique order book shapshots from ./data/hft_data/AAPL/AAPL_2024-03-08_34200000_57600000_message_10.csv


In [3]:
def prep_data(df) -> pd.DataFrame:
    df = df[['Price', 'Size']]
    df.head()

    # sample every 100ms, and the size would be the sum of the size in that 100ms. 
    # Price would be the average price in that 100ms.
    df = df.resample('100ms').agg({'Price': 'mean', 'Size': 'sum'})

    # Check for NaN values

    # forwardfill all NaN values in the data
    df = df.ffill()

    # normalize the data with mean and std
    mean = df['Price'].mean()
    std = df['Price'].std()
    df['Price'] = (df['Price'] - mean) / std

    mean = df['Size'].mean()
    std = df['Size'].std()
    df['Size'] = (df['Size'] - mean) / std

    print("original shape: ", df.shape)

    df = df.values
    # Create a tensor for every 30 minutes of data
    tensors = []
    for i in range(0, len(df), 18000):
        if i + 18000 < len(df):
            # flip the first and second dimension, so that the shape is (batch_size, channel, sequence_length)
            tensors.append(torch.tensor(df[i:i+18000], dtype = torch.float32).unsqueeze(0))
        else:
            tensors.append(torch.tensor(df[i:], dtype = torch.float32).unsqueeze(0))

    return tensors

    # Create the final torch tensor, every 1 hour is a sequence

tensors_list = []
for df in df_list[1:]: 
    tensors_list.extend(prep_data(df))


original shape:  (234000, 2)
original shape:  (234000, 2)
original shape:  (234000, 2)
original shape:  (234000, 2)
original shape:  (234000, 2)


In [4]:
# filp the first and second dimension, so that the shape is (batch_size, channel, sequence_length)
tensors_list = [tensor.permute(0, 2, 1) for tensor in tensors_list]

_ = [print(tensor.shape) for tensor in tensors_list]


torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([1, 2, 18000])
torch.Size([

In [5]:
# Start the training process with tensors.

if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using device {device}")

model = TCNAutoencoder(input_dim=(2, 18000)).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

import tqdm

# Randomly sample 0.2 of the data from the batch for testing, excluding them for traning.
tensors = tensors_list
tensors_train = tensors[:int(len(tensors) * 0.8)]
tensors_test = tensors[int(len(tensors) * 0.8):]

Using device mps


In [6]:
# Train the model

import time
losses = []
for epoch in range(100):
    # time each epoch
    start_time = time.time()
    
    # pack the data into one tensor
    data = torch.cat(tensors_train, dim=0).to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, data)
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch}, Loss: {loss.item()}")
    losses.append(loss.item())
    print(f"Time taken (seconds): {time.time() - start_time}")

    

/Users/chan/.pyenv/versions/3.11.8/lib/python3.11/site-packages/torch/nn/modules/conv.py:306: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/Convolution.cpp:1041.)
  return F.conv1d(input, weight, bias, self.stride,


Epoch: 0, Loss: 1.0067387819290161
Time taken (seconds): 1.9794092178344727
Epoch: 1, Loss: 1.0037273168563843
Time taken (seconds): 1.251244068145752
Epoch: 2, Loss: 1.001622200012207
Time taken (seconds): 1.2493751049041748
Epoch: 3, Loss: 1.00016188621521
Time taken (seconds): 1.243300199508667
Epoch: 4, Loss: 0.9992467164993286
Time taken (seconds): 1.2446019649505615
Epoch: 5, Loss: 0.9985362887382507
Time taken (seconds): 1.2650761604309082
Epoch: 6, Loss: 0.9978350400924683
Time taken (seconds): 1.2596120834350586
Epoch: 7, Loss: 0.9967426657676697
Time taken (seconds): 1.2553818225860596
Epoch: 8, Loss: 0.9949354529380798
Time taken (seconds): 1.2501320838928223
Epoch: 9, Loss: 0.9921074509620667
Time taken (seconds): 1.272416114807129
Epoch: 10, Loss: 0.9880087375640869
Time taken (seconds): 1.2580909729003906
Epoch: 11, Loss: 0.982081949710846
Time taken (seconds): 1.2839579582214355
Epoch: 12, Loss: 0.9737968444824219
Time taken (seconds): 1.2593021392822266
Epoch: 13, Loss:

In [ ]:
# Do 400 more epochs

for epoch in range(900):
    # time each epoch
    start_time = time.time()
    
    # shuffle the data
    np.random.shuffle(tensors_train)

    # pack the data into one tensor
    data = torch.cat(tensors_train, dim=0).to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, data)
    loss.backward()
    optimizer.step()
    print(f"Epoch: {100 + epoch}, Loss: {loss.item()}")
    losses.append(loss.item())
    print(f"Time taken (seconds): {time.time() - start_time}")

    